In [1]:
from lib.utils import *
from lib.models import *
from lib.ekyn import *
from lib.env import *
from lib.datasets import *

In [9]:
class Gandalf(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.encoder = Frodo(n_features=5000,device=DEVICE).to(DEVICE)
        self.lstm = nn.LSTM(16,32,bidirectional=True)
        self.fc1 = nn.Linear(64,3)
    def forward(self,x_2d,classification=True):
        x_2d = x_2d.view(-1,9,1,5000)
        x = torch.Tensor().to(DEVICE)
        for t in range(x_2d.size(1)):
            xi = self.encoder(x_2d[:,t,:,:],classification=False)
            x = torch.cat([x,xi.unsqueeze(0)],dim=0)
        out,_ = self.lstm(x)
        if(classification):
            x = self.fc1(out[-1])
        else:
            x = out[-1]
        return x
model = Gandalf()
model.load_state_dict(torch.load(f'gandalf_spindle_fold_0/best_model.pt'))
model.to(DEVICE)
criterion = torch.nn.CrossEntropyLoss()

In [7]:
FOLD = 0
ids = ['A1','A2','A3','A4','B1','B2','B3','B4','C1','C2','C3','C4','C5','C6','C7','C8','D1','D2','D3','D4','D5','D6']
test_id = ids[FOLD]
ids.remove(test_id)
subjects = [load_spindle_eeg_label_pair(cohort=id[0],subject=id[1]) for id in [test_id]]
Xs = [subject[0] for subject in subjects]
ys = [subject[1] for subject in subjects]
devloader = DataLoader(dataset=SSDataset(Xs,ys,range(8640)),batch_size=32,shuffle=False)

Extracting EDF parameters from /home/andrew/sleep/data/spindle/CohortD/recordings/D6.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [10]:
evaluate(devloader,model,criterion,DEVICE)

100%|██████████| 270/270 [00:22<00:00, 11.88it/s]


(0.10503444508153566,
 {'precision': 0.8851501768842093,
  'recall': 0.9438842513114314,
  'f1': 0.9096107987033584},
 tensor([1., 1., 1.,  ..., 2., 2., 2.]),
 tensor([1., 1., 1.,  ..., 2., 2., 2.]),
 tensor([[1.9276e-04, 9.3401e-01, 6.5802e-02],
         [2.7791e-04, 9.2710e-01, 7.2618e-02],
         [4.9142e-04, 8.4240e-01, 1.5711e-01],
         ...,
         [3.0795e-04, 4.8863e-03, 9.9481e-01],
         [3.6640e-04, 1.8196e-02, 9.8144e-01],
         [4.6634e-04, 1.5415e-01, 8.4538e-01]]))